# Advanced NLP Workshop

In this workshop, you will learn a number of advanced NLP techniques, with exercises based on a real-world dataset of tweets on #Telstra and #Optus.

**Installing required libraries.**

Install pyLDAvis library for interactive topic visualization dashboard.

In [89]:
!pip install pyLDAvis==2.1.2

Load libraries.

In [90]:
# Loading primary libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# For text processing
import re
from textblob import TextBlob
import gensim
import logging
import tempfile
from gensim import corpora, models, similarities
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis.gensim

## Load data

Load the two datasets (Download from LMS)

In [91]:
df_telstra = pd.read_excel("tp7_telco.xlsx", sheet_name='telstra')
df_optus = pd.read_excel("tp7_telco.xlsx", sheet_name='optus')

In [92]:
# display the column names of the datasets
df_optus.columns

Index(['date', 'content', 'likes', 'retweets', 'authorID', 'name', 'gender',
       'followers', 'following', 'nTweets', 'location', 'tweets', 'day'],
      dtype='object')

In [93]:
df_telstra.columns

Index(['date', 'content', 'likes', 'retweets', 'tweets', 'authorID', 'name',
       'gender', 'followers', 'following', 'nTweets', 'location', 'day'],
      dtype='object')

As the first step, we can select a subset of columns required for our analysis. This can be done based on your business requirement.


In [94]:
df_telstra = df_telstra[['day', 'content']]
df_optus = df_optus[['day', 'content']]

In [95]:
df_optus.head()

,day,content
0,2016-01-01,optus ill giv them a try wen i get home is tha...
1,2016-01-01,optus yeah that'll have to do
2,2016-01-01,optus hi are there problems with cable interne...
3,2016-01-01,optus ->ai
4,2016-01-01,optus /waves to esther y


In [96]:
df_telstra.head()

,day,content
0,2016-01-01,telstra runkeeper i went out for a run.
1,2016-01-01,telstra you too :)
2,2016-01-01,telstra here we are no voice service until at ...
3,2016-01-01,home internet down since yesterday. telstra sa...
4,2016-01-01,three times telstra has said there was an erro...


## Preprocessing

Preprocessing applies to both the datasets.  Instead of duplicating the worm ( as we have to do each step twice (for 2 datasets)), we can combine the two datasets, while keeping track of the original dataset. For that, we create a seperate column ('flag') in both datasets, named 'provider' to indicate the source dataset.

In [97]:
# Create a new flag column for both datasets
df_telstra['provider'] = 'telstra'
df_optus['provider'] = 'optus'

In [98]:
df_telstra.head()

,day,content,provider
0,2016-01-01,telstra runkeeper i went out for a run.,telstra
1,2016-01-01,telstra you too :),telstra
2,2016-01-01,telstra here we are no voice service until at ...,telstra
3,2016-01-01,home internet down since yesterday. telstra sa...,telstra
4,2016-01-01,three times telstra has said there was an erro...,telstra


In [99]:
# combine the 2 datasets
df = pd.concat([df_telstra, df_optus], ignore_index=True)

Now we run the preprocessing.

In [100]:
# Transform sentences into lowercase
df['content'] = df['content'].str.lower()

In [101]:
# Following command is just to visualize the processed dataframe
df.tail()

,day,content,provider
37587,2016-02-29,optus yes and every day since. i have been run...,optus
37588,2016-02-29,optus hi i've been having trouble with my 4g f...,optus
37589,2016-02-29,optus clever,optus
37590,2016-02-29,optus your store was not aware of the $40 deal...,optus
37591,2016-02-29,jozmosis optus that would disqualify a lot of ...,optus


Remove twitter user ids that are mentioned in the tweet text. We will use [regular expressions ](https://www.w3schools.com/python/python_regex.asp) to do this.  
In applying the regular expression, we use the string in the format r"regular-expression". This is to treat the regex as a raw string. Additional information on this can be found from [this stackoverflow question](https://stackoverflow.com/questions/4780088/what-does-preceding-a-string-literal-with-r-mean). 

In [102]:
def remove_twitter_ids(tweet):
  mention_removed_tweet = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", str(tweet)) # remove any sequence of characters followed by '@' sign
  spaces_removed = re.sub(r"\s\s+", " ", str(mention_removed_tweet)) # remove multiple spaces
  return spaces_removed

df['content'] = df['content'].apply(remove_twitter_ids)
df.tail()

,day,content,provider
37587,2016-02-29,optus yes and every day since i have been runn...,optus
37588,2016-02-29,optus hi ive been having trouble with my 4g fo...,optus
37589,2016-02-29,optus clever,optus
37590,2016-02-29,optus your store was not aware of the 40 deal ...,optus
37591,2016-02-29,jozmosis optus that would disqualify a lot of ...,optus


In [103]:
# How does above function works?
# Take any tweet that has mentions in it. e.g., https://twitter.com/DavidLKeating/status/1251253645431144457
twt = "Canada just ordered #masks4all for flights. We need this on all US flights NOW. What are we waiting for @SecElaineChao @AmericanAir @Delta @SouthwestAir @united @AlaskaAir @FAANews ??? @jeremyphoward update"
out_twt = remove_twitter_ids(twt)
print(out_twt)

Canada just ordered masks4all for flights We need this on all US flights NOW What are we waiting for update


### Duplicate removal

It is essential we check for duplicates. Because, the tweets are extracted from Twitter API and it is likely the same tweet repeats multiple times.  
We can utilize pandas drop_duplicates() function. [Link to API](https://pandas.pydata.org/pandas-docs/version/0.24.2/reference/api/pandas.DataFrame.drop_duplicates.html)

In [104]:
# Check duplicate tweets count
duplicate_count = len(df['content'])-len(df['content'].drop_duplicates())
print('duplicate count:', duplicate_count)
print('total records before remove duplicates:', df.shape[0])

duplicate count: 1215
total records before remove duplicates: 37592


In [105]:
# drop duplicates (keep the last tweet of each of the duplicates)
df = df.drop_duplicates(subset='content', keep="first")
print('updated record count:', df.shape[0])

updated record count: 36377


Now we will remove all punctuation, retaining just the text.




In [106]:
# Remove punctuations 
df['content'] = df['content'].str.replace(r'[^\w\s]','')  # This is the use of regular expressions.
df.tail()

,day,content,provider
37587,2016-02-29,optus yes and every day since i have been runn...,optus
37588,2016-02-29,optus hi ive been having trouble with my 4g fo...,optus
37589,2016-02-29,optus clever,optus
37590,2016-02-29,optus your store was not aware of the 40 deal ...,optus
37591,2016-02-29,jozmosis optus that would disqualify a lot of ...,optus


### Stop words and domain related word removal

Now let's conduct further text pre- processing using NLTK library as we did in the first NLP workshop. First, remove stopwords.

In [107]:
#Remove stop words
# Load NLTK library
import nltk

# Download the stopwords to the nltk library
nltk.download('stopwords')

# Load the stopwords
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [108]:
# get the list of all stopwords from the library
stop = stopwords.words('english')

In [109]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Note that the term 'not' is in the stop word list. This will have a negative impact on sentiment analysis. However, if the NLP technique is topic modeling,  there will be no such impact. It is important to keep this mind, drop or retain words depending on the goal of your exercise.  
For this case, we will remove the term 'not' from the stopword list.

In [110]:
stop.remove('not')

In [111]:
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', '

In [112]:
# Remove the words in 'stop' list
def remove_stop_words(tweet):
  tokens = tweet.split()
  stop_removed_tokens = [t for t in tokens if t not in stop]
  convert_to_string = " ".join(stop_removed_tokens)
  return convert_to_string
  
df['content'] = df['content'].apply(remove_stop_words)
df.tail(5)

,day,content,provider
37587,2016-02-29,optus yes every day since running isolation te...,optus
37588,2016-02-29,optus hi ive trouble 4g last days dont know iv...,optus
37589,2016-02-29,optus clever,optus
37590,2016-02-29,optus store not aware 40 deal 12 month byo dea...,optus
37591,2016-02-29,jozmosis optus would disqualify lot push notif...,optus


In [113]:
# How above function works?
def remove_stop_words(tweet):
  tokens = tweet.split()
  stop_removed_tokens = [t for t in tokens if t not in stop]
  convert_to_string = " ".join(stop_removed_tokens)
  return convert_to_string

txt = "you are the first person."
remove_stop_words(txt)

'first person.'

In [114]:
# For your knowledge: Same above function can be written as a lambda function
# df['content'] = df['content'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

### Standardization of tweets

We shall use stemming to standardize tweets.  
[Stemming](https://en.wikipedia.org/wiki/Stemming) is the removal of prefix, suffix etc, to derive the base form of a word. We will use the NLTK library.

In [115]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stemming_function(sent):
  word_list = sent.split()
  stemmed_word_list = [stemmer.stem(word) for word in word_list]
  stemmed_sentence = " ".join(stemmed_word_list)
  return stemmed_sentence

df['content_stem'] = df['content'].apply(stemming_function)

In [116]:
# Compare the content vs. stemmed content
df[['content', 'content_stem']].tail(10)

,content,content_stem
37582,optus net pisses,optu net piss
37583,bulldogchris gus10gould juzweezy case point lo...,bulldogchri gus10gould juzweezi case point loo...
37584,leaveforthecup optus yeah good luck,leaveforthecup optu yeah good luck
37585,dragonsfan1988 gus10gould juzweezy foxtel dead...,dragonsfan1988 gus10gould juzweezi foxtel dead...
37586,dragonsfan1988 gus10gould foxtel overpriced nr...,dragonsfan1988 gus10gould foxtel overpr nrl re...
37587,optus yes every day since running isolation te...,optu ye everi day sinc run isol test 2 week pl...
37588,optus hi ive trouble 4g last days dont know iv...,optu hi ive troubl 4g last day dont know ive p...
37589,optus clever,optu clever
37590,optus store not aware 40 deal 12 month byo dea...,optu store not awar 40 deal 12 month byo deal ...
37591,jozmosis optus would disqualify lot push notif...,jozmosi optu would disqualifi lot push notif g...


The stemming has turned term 'optus' to 'optu', thus, stemming has not deemed expected results in this context.  Thereby, we will go ahead with original content without any standardization.   
You may attempt to lemmatize and see if you can improve the results.

## Word frequency analysis

Let us look at the most occuring words and the least occuring words in our tweet dataset.

In [117]:
# Create a word frequency series. (This is a pandas series)
word_frequency = pd.Series(' '.join(df['content']).split()).value_counts()

In [118]:
# Look at the top 10 words (you can write either word_frequency[:10] or word_frequency[0:10]. Both give the same result.)
word_frequency[:10]

telstra     27434
optus       12871
not          3994
internet     3190
phone        2775
get          2668
service      2641
im           2258
thanks       2210
call         1992
dtype: int64

In [ ]:
# Visualize the top word counts 

word_count  = word_frequency
word_count = word_count[:10,]
plt.figure(figsize=(10,5))
sns.barplot(word_count.index, word_count.values, alpha=0.8)
plt.title('Tweets in top 10 words')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Word', fontsize=12)
plt.show()

We will now visualize the text corpus that we created after pre-processing to get insights on the most frequently used words.

In [120]:
from PIL import Image
from wordcloud import WordCloud

In [ ]:
corpus = list(df['content'])

wordcloud = WordCloud(background_color='white', max_words=200, max_font_size=50, random_state=42).generate(str(corpus))

fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

**Insight**  
From the word cloud we could identify that terms telstra and optus has been mentioned frequently. But this is expected - not so much an insight. Therefore, you may remove high frequent yet non-insightful words alongside the twitter specific words.

### Remove common words

Let's  remove context specific words and the most common words from the tweets.

The tweets extracted are from the hashtags @telstra and @optus, these keywords will appear in most tweets. Let's  add those keywords to the set of words to be removed.  
We can extend the new_words list with those additional keywords, which you may think will be widely used due to the area/domain the tweets were extracted.

In [122]:
# Creating a list of custom stopwords
new_words_to_remove = ["pic", "twitter", "com", "telstra", "optus"]

In [123]:
# Remove common words
# We will use lambda function here.
df['content'] = df['content'].apply(lambda x: " ".join(x for x in x.split() if x not in new_words_to_remove))
df.head(5)

,day,content,provider,content_stem
0,2016-01-01,runkeeper went run,telstra,telstra runkeep went run
1,2016-01-01,,telstra,telstra
2,2016-01-01,voice service least mon box door still open,telstra,telstra voic servic least mon box door still open
3,2016-01-01,home internet since yesterday says fixed 6pm 4...,telstra,home internet sinc yesterday telstra say fix 6...
4,2016-01-01,three times said error porting said completed ...,telstra,three time telstra said error port said comple...


Exercise: try to replicate lambda function above as a python function.

We will re-attempt to compose a frequency appearing word list.

In [124]:
# Create a word frequency series. (This is a pandas series)
word_frequency = pd.Series(' '.join(df['content']).split()).value_counts()

In [125]:
# Look at the top 10 words (you can write either word_frequency[:10] or word_frequency[0:10]. Both give the same result.)
word_frequency[:10]

not         3994
internet    3190
phone       2775
get         2668
service     2641
im          2258
thanks      2210
call        1992
still       1968
data        1761
dtype: int64

In [ ]:
corpus = list(df['content'])

wordcloud = WordCloud(background_color='white', max_words=200, max_font_size=50, random_state=42).generate(str(corpus))

fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## Bigrams and Trigrams

The following commands will extract bigrams from the tweet dataset.  
From here onwards, we will isolate tweets for each provider to conduct a comparative analysis.

In [127]:
# Seperate the two datasets by using pandas filtering mechanism.
df_telstra_processed = df.loc[df['provider'] == 'telstra']
df_optus_processed = df.loc[df['provider'] == 'optus']

### Bigrams

We will use CountVectorizer object from sklearn to generate bi-grams and tri-grams. [Link to API](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)  

You may refer [this StackoverFlow explanation](https://stackoverflow.com/questions/24005762/understanding-the-ngram-range-argument-in-a-countvectorizer-in-sklearn) to understand ngram_range parameter.

In [128]:
# This function will generate most frequently occuring Bi-grams
def get_ngrams(corpus, ngram_range=(2, 2)):
    
    # Create CountVectorizer object from sklearn library with bigrams
    vec1 = CountVectorizer(ngram_range=ngram_range, max_features=2000).fit(corpus)

    # Create BoW feature representation using word frequency
    bag_of_words = vec1.transform(corpus)

    # compute sum of words
    sum_words = bag_of_words.sum(axis=0) 

    # create (word, frequency) tuples for bigrams
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq

Try out an example to see how this  function works.

In [129]:
txts = ['John likes to watch movies', 'Mary likes movies too', 'Mary also likes to watch football games']
get_ngrams(txts, ngram_range=(4, 4))

[('john likes to watch', 1),
 ('likes to watch movies', 1),
 ('mary likes movies too', 1),
 ('mary also likes to', 1),
 ('also likes to watch', 1),
 ('likes to watch football', 1),
 ('to watch football games', 1)]

In [130]:
bigrams_telstra = get_ngrams(df_telstra_processed['content'].tolist(), ngram_range=(2, 2))
bigrams_optus = get_ngrams(df_optus_processed['content'].tolist(), ngram_range=(2, 2))

In [131]:
bigrams_telstra

[('free data', 441),
 ('customer service', 275),
 ('call back', 186),
 ('not working', 182),
 ('im not', 179),
 ('data day', 136),
 ('still waiting', 131),
 ('still not', 128),
 ('mobile data', 125),
 ('cable internet', 117),
 ('phone line', 116),
 ('call centre', 115),
 ('home phone', 101),
 ('dont know', 97),
 ('internet connection', 93),
 ('tech support', 93),
 ('live chat', 91),
 ('cant get', 89),
 ('looks like', 88),
 ('case manager', 88),
 ('not sure', 88),
 ('new phone', 84),
 ('24 hours', 82),
 ('not good', 82),
 ('last week', 82),
 ('service status', 81),
 ('cant even', 81),
 ('last night', 78),
 ('internet service', 74),
 ('whats going', 74),
 ('not happy', 73),
 ('phone calls', 67),
 ('doesnt work', 65),
 ('not even', 64),
 ('reference number', 63),
 ('dont want', 63),
 ('phone call', 60),
 ('data today', 60),
 ('im still', 58),
 ('status page', 57),
 ('phone internet', 57),
 ('isnt working', 57),
 ('new modem', 54),
 ('get free', 54),
 ('trying get', 54),
 ('unlimited data'

Convert bigrams of both datasets to a dataframe with column names bi-gram and frequency.

In [132]:
bigrams_telstra_df = pd.DataFrame(bigrams_telstra)
bigrams_telstra_df.columns=["Bi-gram", "Freq"]

bigrams_optus_df = pd.DataFrame(bigrams_optus)
bigrams_optus_df.columns=["Bi-gram", "Freq"]

In [ ]:
# Barplot of most freq Bi-grams
top_bigrams_to_show = 20

sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Bi-gram", y="Freq", data=bigrams_telstra_df[:top_bigrams_to_show])
h.set_xticklabels(h.get_xticklabels(), rotation=45)  # here rotation parameter shows the angle of your x-axis labels
plt.title('Telstra - Bigram Analysis')
plt.show()

**Insights**:  
The bigram 'data day' refers to an event. After a series of outages, Telstra declared a free data day as a compensation. Notice how bigrams can be used to detect such events as well as topics/themes specific to the domain.

In [ ]:
# Barplot of most freq Bi-grams for optus
top_bigrams_to_show = 20

sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Bi-gram", y="Freq", data=bigrams_optus_df[:top_bigrams_to_show])
h.set_xticklabels(h.get_xticklabels(), rotation=45)
plt.title('Optus - Bigram Analysis')
plt.show()

**Insights:**  
Observe that in Optus dataset, there are many words related to sport which were not prominent in Telstra data. Bigram frequencies can be used to differentiate and compare topics of interest.

### Trigrams

Let's now attempt trigrams. 

In [135]:
trigrams_telstra = get_ngrams(df_telstra_processed['content'].tolist(), ngram_range=(3, 3))
trigrams_optus = get_ngrams(df_optus_processed['content'].tolist(), ngram_range=(3, 3))

Convert tri-grams of both datasets to a dataframe with column names tri-gram and frequency.

In [136]:
trigrams_telstra_df = pd.DataFrame(trigrams_telstra)
trigrams_telstra_df.columns=["Tri-gram", "Freq"]

trigrams_optus_df = pd.DataFrame(trigrams_optus)
trigrams_optus_df.columns=["Tri-gram", "Freq"]

In [ ]:
# Barplot of most freq Tri-grams
top_trigrams_to_show = 20

sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Tri-gram", y="Freq", data=trigrams_telstra_df[:top_trigrams_to_show])
h.set_xticklabels(h.get_xticklabels(), rotation=90)
plt.title('Telstra - Trigram Analysis')
plt.show()

In [ ]:
# Barplot of most freq Tri-grams
top_trigrams_to_show = 20

sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x="Tri-gram", y="Freq", data=trigrams_optus_df[:top_trigrams_to_show])
h.set_xticklabels(h.get_xticklabels(), rotation=90)
plt.title('Optus - Tri Analysis')
plt.show()

Can you determine new insights from Trigram plots?

## Temporal Analysis

Now let's attempt to explore temporal patterns in the two datasets.

In [139]:
# Group the twitter datasets based on the date field and get the count per each day.
df_telstra_date_wise = df_telstra_processed.groupby(['day'])['content'].count().reset_index(name='tweet_count_telstra').set_index('day')
df_optus_date_wise = df_optus_processed.groupby(['day'])['content'].count().reset_index(name='tweet_count_optus').set_index('day')

In [140]:
df_telstra_date_wise.head()

,tweet_count_telstra
day,
2016-01-01,147
2016-01-02,518
2016-01-03,418
2016-01-04,696
2016-01-05,257


In [141]:
df_optus_date_wise.tail()

,tweet_count_optus
day,
2016-02-25,160
2016-02-26,220
2016-02-27,140
2016-02-28,100
2016-02-29,155


When plotting with python, we can change the size of the plot and save it as a figure in your workspace.  
* Change size: plt.figure(figsize=(width, length)) # width and length are in inches  
* Save the image: plt.savefig(filename, dpi)  # dpi is the resolution in pixels

In [ ]:
ax = df_telstra_date_wise.plot()
df_optus_date_wise.plot(ax=ax)
plt.savefig('temporal_analysis_v2.png', dpi=800)

Output image is saved in files tab. You can right click on the image and download it to your computer for further analysis.

Insights: How would you describe the two line plots? Any actionable insights?  
Exerciese: Go through the actual Telstra dataset on 2018-01-28, can you explain this spike in tweet count?

In [ ]:
# this is a helper code if you have 3 or more dataframes to compare with.
# df_ABC_date_wise =df_optus_date_wise.copy()
# df_ABC_date_wise['tweet_count_ABV'] = df_ABC_date_wise['tweet_count_optus'] + 20
# del df_ABC_date_wise['tweet_count_optus']

ax = df_telstra_date_wise.plot()
df_optus_date_wise.plot(ax=ax)
# df_ABC_date_wise.plot(ax=ax)
plt.savefig('temporal_analysis_v3.png', dpi=800)

## Sentiment Analysis

Analyse sentiments for each tweet, aggregate into the sentiment of particular day and derive a sentiment timeline.

In [144]:
# Extract sentiments from the tweets (This should take some time because we run this to the entire dataset.)
df['sentiment'] = df['content'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [145]:
# Display first 50 tweets with respective sentiment value
df[['content', 'sentiment']].head(50)

,content,sentiment
0,runkeeper went run,0.000000
1,,0.000000
2,voice service least mon box door still open,-0.150000
3,home internet since yesterday says fixed 6pm 4...,0.100000
4,three times said error porting said completed ...,0.000000
5,last six weeks ed continually porting number a...,0.000000
6,already,0.000000
7,per tweet service restored 6pm 4th jan ridiculous,-0.333333
8,bilis nang hahahaha,0.200000
9,awesome thanks nye text im worried im missing ...,0.220000


Again we will conduct invididual analysis for both telcom providers.

In [146]:
df_telstra_processed = df.loc[df['provider'] == 'telstra']
df_optus_processed = df.loc[df['provider'] == 'optus']

Aggregate the sentiment value (using mean) for each provider

In [147]:
df_telstra_processed

,day,content,provider,content_stem,sentiment
0,2016-01-01,runkeeper went run,telstra,telstra runkeep went run,0.000
1,2016-01-01,,telstra,telstra,0.000
2,2016-01-01,voice service least mon box door still open,telstra,telstra voic servic least mon box door still open,-0.150
3,2016-01-01,home internet since yesterday says fixed 6pm 4...,telstra,home internet sinc yesterday telstra say fix 6...,0.100
4,2016-01-01,three times said error porting said completed ...,telstra,three time telstra said error port said comple...,0.000
...,...,...,...,...,...
25567,2016-02-29,stop mobile redirecting home phone windows pho...,telstra,telstra stop mobil redirect home phone window ...,0.000
25568,2016-02-29,gg ed thegmunay,telstra,gg telstra ed thegmunay,0.000
25569,2016-02-29,get shit together ffs twice today internet died,telstra,telstra get shit togeth ff twice today interne...,-0.200
25570,2016-02-29,yeh technical error explanation told one date ...,telstra,telstra yeh technic error explan told one date...,0.125


In [148]:
df_telstra_processed.groupby(['day'])['sentiment'].mean()

day
2016-01-01    0.003838
2016-01-02    0.028647
2016-01-03    0.066871
2016-01-04    0.043939
2016-01-05    0.032759
2016-01-06    0.009095
2016-01-07    0.016708
2016-01-08    0.031807
2016-01-09    0.038777
2016-01-10    0.060581
2016-01-11    0.034409
2016-01-12    0.013710
2016-01-13    0.010740
2016-01-14    0.021275
2016-01-15    0.012664
2016-01-16    0.038314
2016-01-17   -0.003051
2016-01-18    0.046497
2016-01-19    0.033564
2016-01-20    0.037302
2016-01-21    0.058624
2016-01-22    0.031233
2016-01-23    0.054891
2016-01-24    0.005957
2016-01-25    0.031732
2016-01-26    0.034476
2016-01-27    0.026120
2016-01-28    0.064812
2016-01-29    0.045313
2016-01-30    0.014393
2016-01-31    0.016682
2016-02-01    0.010754
2016-02-02    0.015110
2016-02-03    0.047731
2016-02-04    0.042135
2016-02-05    0.010857
2016-02-06    0.034101
2016-02-07    0.034366
2016-02-08    0.012498
2016-02-09    0.023839
2016-02-10    0.062685
2016-02-11    0.017418
2016-02-12    0.024969
2016-02

In [149]:
df_senti_telstra_date_wise = df_telstra_processed.groupby(['day'])['sentiment'].mean().reset_index(name='mean_sentiment_telstra').set_index('day')
df_senti_optus_date_wise = df_optus_processed.groupby(['day'])['sentiment'].mean().reset_index(name='mean_sentiment_optus').set_index('day')

Plot sentiment over time.

In [ ]:
# Plot twitter sentimet timeline over each provider
ax = df_senti_telstra_date_wise.plot()
df_senti_optus_date_wise.plot(ax=ax)
plt.savefig('temporal_sentiment_v2.png', dpi=800)

**Insights:**   
When deriving insights you can refer to days with significant sentiment scores (peaks) and explore the tweets to identify what event has led to positive/negative sentiment.

## Topic Modeling

We will conduct a topic modeling for one of the twitter datasets here (Telstra).  
It is possible to conduct topic modeling using single words, bigrams, trigrams or n-grams.  
Will be using a topic modeling algorithm named - LDA (Latent Dirichlet Allocation) for this task. 

In [151]:
# Setting up the environment for LDA algorithm.

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Folder "/tmp" will be used to save temporary dictionary and corpus.


In [152]:
# Filter the dataset of the telstra only.
df_telstra_processed = df.loc[df['provider'] == 'telstra']

Now we will create the text corpus.  


In [153]:
# Convert the tweets as the text corpus.
corpus = list(df_telstra_processed['content'])

In [154]:
corpus[:5]

['runkeeper went run',
 '',
 'voice service least mon box door still open',
 'home internet since yesterday says fixed 6pm 4jan five days areyoujoking pleaseexplain',
 'three times said error porting said completed wasnt cancelled']

In [155]:
# Tokenization
telstra_texts = [[word for word in str(document).split()] for document in corpus]

In [156]:
telstra_texts[:5]

[['runkeeper', 'went', 'run'],
 [],
 ['voice', 'service', 'least', 'mon', 'box', 'door', 'still', 'open'],
 ['home',
  'internet',
  'since',
  'yesterday',
  'says',
  'fixed',
  '6pm',
  '4jan',
  'five',
  'days',
  'areyoujoking',
  'pleaseexplain'],
 ['three',
  'times',
  'said',
  'error',
  'porting',
  'said',
  'completed',
  'wasnt',
  'cancelled']]

In [157]:
# Create a dictionary based on the tokanized words of all the tweets.
dictionary = corpora.Dictionary(telstra_texts)

# Save the above dictionary as a local file for LDA model to access.
dictionary.save(os.path.join(TEMP_FOLDER, 'telstra.dict'))

2021-04-12 12:59:10,173 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-04-12 12:59:10,388 : INFO : adding document #10000 to Dictionary(13861 unique tokens: ['run', 'runkeeper', 'went', 'box', 'door']...)
2021-04-12 12:59:10,583 : INFO : adding document #20000 to Dictionary(21290 unique tokens: ['run', 'runkeeper', 'went', 'box', 'door']...)
2021-04-12 12:59:10,700 : INFO : built Dictionary(25075 unique tokens: ['run', 'runkeeper', 'went', 'box', 'door']...) from 25289 documents (total 225870 corpus positions)
2021-04-12 12:59:10,704 : INFO : saving Dictionary object under /tmp/telstra.dict, separately None
/usr/local/lib/python3.7/dist-packages/smart_open/smart_open_lib.py:479: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
2021-04-12 12:59:10,724 : INFO : saved /tmp/telstra.dict


When creating the dictionary, each tweet (document) is alphabetically sorted before composing the dictionary.

In [158]:
print(telstra_texts[0])
print('alphabetically sorted', sorted(telstra_texts[0]))

['runkeeper', 'went', 'run']
alphabetically sorted ['run', 'runkeeper', 'went']


In [159]:
print(telstra_texts[1])
print('alphabetically sorted', sorted(telstra_texts[1]))

[]
alphabetically sorted []


In [160]:
# Print the dictionary
print(dictionary.token2id)

{'run': 0, 'runkeeper': 1, 'went': 2, 'box': 3, 'door': 4, 'least': 5, 'mon': 6, 'open': 7, 'service': 8, 'still': 9, 'voice': 10, '4jan': 11, '6pm': 12, 'areyoujoking': 13, 'days': 14, 'five': 15, 'fixed': 16, 'home': 17, 'internet': 18, 'pleaseexplain': 19, 'says': 20, 'since': 21, 'yesterday': 22, 'cancelled': 23, 'completed': 24, 'error': 25, 'porting': 26, 'said': 27, 'three': 28, 'times': 29, 'wasnt': 30, 'amaysim': 31, 'continually': 32, 'ed': 33, 'joke': 34, 'last': 35, 'number': 36, 'six': 37, 'weeks': 38, 'already': 39, '4th': 40, 'jan': 41, 'per': 42, 'restored': 43, 'ridiculous': 44, 'tweet': 45, 'bilis': 46, 'hahahaha': 47, 'nang': 48, 'awesome': 49, 'friends': 50, 'future': 51, 'get': 52, 'im': 53, 'missing': 54, 'nye': 55, 'offers': 56, 'text': 57, 'thanks': 58, 'worried': 59, 'bigpond': 60, 'bill': 61, 'charged': 62, 'double': 63, 'find': 64, 'ill': 65, 'movie': 66, 'next': 67, '24': 68, 'gone': 69, 'hit': 70, 'hours': 71, 'long': 72, 'something': 73, 'take': 74, 'trans

In [161]:
# Convert the text dictionary to bag of words model
corpus = [dictionary.doc2bow(text) for text in telstra_texts]

Can you explain this output?

In [162]:
tweet_id = 0
print(telstra_texts[tweet_id]) # each tweet converted to tokens
print(dictionary.doc2bow(telstra_texts[tweet_id])) # each token is represented as a id from a dictionary

['runkeeper', 'went', 'run']
[(0, 1), (1, 1), (2, 1)]


### Generate the topic model

Recall we initially loaded models library from gensim.  
(from gensim import corpora, models, similarities)

In [163]:
# Construct TF-IDF features from the dictionary.
tfidf = models.TfidfModel(corpus)

2021-04-12 12:59:11,433 : INFO : collecting document frequencies
2021-04-12 12:59:11,436 : INFO : PROGRESS: processing document #0
2021-04-12 12:59:11,483 : INFO : PROGRESS: processing document #10000
2021-04-12 12:59:11,518 : INFO : PROGRESS: processing document #20000
2021-04-12 12:59:11,542 : INFO : calculating IDF weights for 25289 documents and 25074 features (220624 matrix non-zeros)


In [164]:
# Transform the tweets as TF-IDF feature vectors
corpus_tfidf = tfidf[corpus] 

We need to define how many topics we capture through LDA.

In [165]:
total_topics = 40

Below code will build the LDA topic model.   
We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.



In [166]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.  
You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics() as shown next.





In [167]:
# Print the Keyword in the 10 topics
lda.show_topics(total_topics, num_words=6)

[(0,
  '0.072*"think" + 0.031*"presale" + 0.027*"might" + 0.022*"happened" + 0.022*"system" + 0.020*"thought"'),
 (1,
  '0.064*"area" + 0.032*"internet" + 0.023*"goes" + 0.022*"supposed" + 0.021*"steph" + 0.019*"dropped"'),
 (2,
  '0.021*"name" + 0.021*"yyy" + 0.020*"several" + 0.019*"sold" + 0.019*"late" + 0.018*"n"'),
 (3,
  '0.035*"cannot" + 0.034*"answer" + 0.031*"text" + 0.028*"connect" + 0.028*"exchange" + 0.021*"melbourne"'),
 (4,
  '0.037*"number" + 0.035*"home" + 0.033*"getting" + 0.025*"service" + 0.018*"mobile" + 0.017*"not"'),
 (5,
  '0.061*"ill" + 0.039*"try" + 0.037*"give" + 0.026*"something" + 0.025*"understand" + 0.018*"running"'),
 (6,
  '0.031*"12" + 0.030*"not" + 0.028*"check" + 0.022*"network" + 0.021*"point" + 0.021*"ask"'),
 (7,
  '0.064*"adsl" + 0.041*"outage" + 0.026*"order" + 0.021*"signal" + 0.021*"speak" + 0.020*"22"'),
 (8,
  '0.028*"hours" + 0.028*"still" + 0.027*"phone" + 0.026*"told" + 0.023*"2" + 0.022*"waiting"'),
 (9,
  '0.038*"seems" + 0.032*"tech" + 

How will you interpret this?  
Assume a topic is a represented as '0.020*"nbn" + 0.010*"thanks" + 0.009*"turnbullmalcolm" + 0.009*"mobile" + 0.009*"copper" + 0.008*"buy"'  
  
It means the top 6 keywords that contribute to this topic are: nbn, thanks, turnbullmalcolm, mobile, copper, buy and the weight of ‘nbn’ on topic 5 is 0.020.  
  
The weights reflect how important a keyword is to that topic.  

Looking at these keywords, can you guess what this topic could be? You may summarise it as nbn connectivity (or more specifically Turnbull's decision as Telecommunications Minister to continue using the copper network for the NBN).  

Likewise, can you go through the remaining keyword subsets and infer each topic?  







### Interactive topic analyzer

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

What is the relevance matrix? (The slider on the right)  
Understand what it means and how it can be used to derive more relevant topics from [this paper](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf).

## References

1.   [LDA Topic Modeling](https://youtu.be/3mHy4OSyRf0)
2.   [Topic Modelling in Python - Tutorial](https://ourcodingclub.github.io/2018/12/10/topic-modelling-python.html)